# WEBSITE BROCHURE

An AI-powered brochure generator. It uses a Large Language Model (LLM) to create customized brochure content based on user input about website link . The current version runs entirely in a notebook, and I plan to convert it into a full website with a simple, user-friendly UI.

In [1]:
# Importing all the libraries and functions
import json
from scraper import fetch_website_contents, fetch_website_links
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import os

In [2]:
load_dotenv(override = True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and len(api_key)>10 and api_key.startswith('sk-proj'):
    print("Everything is good to go")
else:
    print("Error came")
MODEL = 'gpt-5-nano'
openai = OpenAI()

Everything is good to go


In [3]:
links = fetch_website_links("https://edwarddonner.com/")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.
    Links (some might be relative links):
    """
    links = fetch_website_links(url)
    user_prompt +="\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com/"))


    Here is the list of links on the website https://edwarddonner.com/ -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.
    Links (some might be relative links):
    https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-

In [14]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {'role': "system", 'content': link_system_prompt},
            {'role': 'user', 'content': get_links_user_prompt(url)}
        ],
        response_format = {'type':'json_object'}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links 


In [15]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'Nebula page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

### Creating the brochure

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [18]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:2_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure('Hugging Face','https://huggingface.co/')

# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant AI and machine learning community and platform, dedicated to shaping the future of artificial intelligence through collaboration and open innovation. It serves as a central hub where machine learning practitioners, researchers, and enthusiasts come together to create, share, and improve models, datasets, and applications.

---

## What We Offer

### Extensive Model Repository
- Access and browse **over 1 million machine learning models**.
- Includes trending models updated frequently, such as Tongyi-MAI/Z-Image-Turbo, facebook/sam3, and others.
- Models span across diverse use cases including vision, language, video generation, and more.

### Rich Dataset Collection
- Explore **more than 250,000 datasets**, catering to various AI tasks like autonomous vehicles, language modeling, and audio processing.
- Constantly updated datasets from both community members and industry leaders like NVIDIA.

### Spaces — Collaborative AI Applications
- Run, develop, and share AI applications in an interactive environment.
- Featured Spaces include tools for image generation from text prompts, video creation, training guides, and editing.
- Supports seamless deployment and sharing, promoting innovation and experimentation.

### Cutting-edge Open Source Stack
- Hugging Face provides an open source ecosystem allowing creators and enterprises to move faster with proven tools and frameworks.
- Designed to empower both individual developers and businesses to deploy world-class machine learning solutions.

---

## Company Culture

- **Community-driven:** Hugging Face thrives on collaboration and openness, encouraging contributions from developers worldwide.
- **Innovative & Inclusive:** By democratizing AI, the platform fosters innovation across diverse fields and welcomes participation from all experience levels.
- **Empowerment:** The company enables users to create freely without limits on public sharing and collaboration.
- **Transparency:** Open source principles guide the company culture, building trust and accelerating AI advancement.

---

## Customers & Users

- Individual researchers and ML practitioners seeking accessible, high-quality models and datasets.
- Enterprises leveraging ML to build innovative AI-powered products and solutions.
- Academic institutions and educators using Hugging Face resources for teaching and research.
- Developers aiming to deploy fast and efficient AI applications via Spaces.

---

## Careers at Hugging Face

Join the cutting edge of AI research and product development by becoming part of Hugging Face. The team is passionate about open collaboration and building tools that empower the global AI community. Roles typically focus on:
- Machine Learning Research and Engineering
- Software Development
- Community and Developer Relations
- Enterprise Solutions and Customer Success

Hugging Face fosters a supportive, inclusive environment where diverse talents contribute to advancing AI technology and accessibility.

---

## Why Choose Hugging Face?

- **Largest community-driven ML platform** continuously evolving with user needs.
- **Open collaboration** enables faster innovation and knowledge sharing.
- **Comprehensive resources** from models to datasets to applications all integrated in one platform.
- **Focus on accessibility** democratizing access to AI tools for users worldwide.

---

Explore the future of AI —  
**Join Hugging Face today!**  
[Visit Hugging Face](https://huggingface.co) to browse models, datasets, and AI applications or get involved with the community.

---

*Hugging Face: Where AI builders meet, collaborate, and create.*